### IMPORTY  

In [86]:
import queue
import numpy as np
from typing import List, Tuple
import math
import datetime
import pandas as pd
import itertools
from random_gen import RandomNumberGenerator2
import time
from multiprocessing import Pool, cpu_count
import random
import matplotlib.pyplot as plt
import json
import pickle
from tabulate import tabulate


### GENERACJA DANYCH

In [87]:
seed = 289
np.random.seed(seed)
random.seed(seed)
INPUT_SIZE = 100
   
random_gen = RandomNumberGenerator2(seedVaule=seed) # type: ignore
p, d = random_gen.generate_input(INPUT_SIZE)

### ALGORYTM

In [88]:
class AlgorithmSA:
    def __init__(self, 
                 iterations:int,
                 alpha=0.995
                 ):
        self.iterations = iterations
        self.alpha = alpha
        
        
    def eval_end_times(self, p) -> List:
        C = []
        length = p[0].shape[0]
        for i in range(length):
            if i == 0:
                C.append(p[0][i] + p[1][i] + p[2][i])
            else:
                tmp = C[i-1] + p[0][i] + p[1][i] + p[2][i]
                C.append(tmp)
        return C
        
        
    
    def eval_criterion(self, p, d) -> Tuple:
        kry_3 = None
        kry_4 = 0
        kry_5 = None
        kry_6 = 0
        C = self.eval_end_times(p)
        length = p[0].shape[0]
        for i in range(length):
            diff = C[i] - d[i]
            
            if kry_5 is None or kry_5 < diff:
                kry_5 = diff
            
            diff_pos = max(diff, 0)
            if kry_3 is None or kry_3 < diff_pos:
                kry_3 = diff_pos
            kry_4 += diff_pos
            kry_6 += diff                
        return [kry_3, kry_4, kry_5, kry_6]              
        
    
    def get_initial(self, p, d):
        length = len(p[0])
        new_order = np.random.permutation(length)
        p = [p[0][new_order], p[1][new_order], p[2][new_order]]
        d = d[new_order]
        return p, d
    
    def get_neighbor(self, p, d):
        neighbor = p.copy()
        i, j = random.sample(range(p[0].shape[0]), 2)
        neighbor[0][i], neighbor[0][j] = neighbor[0][j], neighbor[0][i]
        neighbor[1][i], neighbor[1][j] = neighbor[1][j], neighbor[1][i]
        neighbor[2][i], neighbor[2][j] = neighbor[2][j], neighbor[2][i]
        
        neighbor_d = d.copy()
        neighbor_d[i], neighbor_d[j] = neighbor_d[j], neighbor_d[i]
        return neighbor, neighbor_d
                        
    def dominates(self, a, b):
        return np.all(b <= a) and np.any(b < a)
    
    def calc_prob(self, it):
        try:
            return math.pow(self.alpha, it)
        except Exception as e:
            return 0.0001
    
    def run(self, p, d):
        P = []
        it = 0
        p, d = self.get_initial(p, d)
        x = self.eval_criterion(p, d)
        P.append(x)
        
        while it < self.iterations:
            p_neigh, d_neigh = self.get_neighbor(p, d)
            x_prim = self.eval_criterion(p_neigh, d_neigh)
            
            if self.dominates(x_prim, x):
                p = p_neigh
                d = d_neigh
                x = x_prim
                P.append(x)
            elif (random.random() < self.calc_prob(it)):
                p = p_neigh
                d = d_neigh
                x = x_prim
                P.append(x)
            it += 1
            
        # Pareto
        F = P.copy()
        for a in range(len(F)):
            for b in range(len(F)):
                if F[a] is not None and F[b] is not None and a != b and self.dominates(F[b], F[a]):
                    F[a] = None
                    break
        F = [f for f in F if f is not None]
        F = np.array(F)
        
        return F, P

### ZAPISANIE WYNIKÓW - WYWOŁANIE ALGORYTMU

In [89]:
reps = 5
REPETITIONS = 1

results = []

# Gathering results for different max iterations
for r in range(reps):
    algorithm = AlgorithmSA(iterations=1600)
    F, P = algorithm.run(p, d)
    results.append((r, F, P))

In [90]:
def create_paths(dir, n: int):
    return f'{dir}/n_{n}_results.pkl', f'{dir}/n_{n}_results_for_hvi.pkl'

DIR = 'res/zad3'
RESULTS_PATH, RESULTS_FOR_HVI_PATH = create_paths(DIR, INPUT_SIZE)

In [91]:
with open(RESULTS_PATH, 'wb') as f:
    pickle.dump(results, f)

### LOAD DATA

In [92]:
with open(RESULTS_PATH, 'rb') as f:
    results_loaded = pickle.load(f)

In [93]:
df = pd.DataFrame(columns=['repetition', 'kry_3', 'kry_4', 'kry_5', 'kry_6'], 
                  data=[(r, *F.mean(axis=0)) for r, F, _ in results_loaded])
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+--------------+---------+---------+---------+---------+
|    |   repetition |   kry_3 |   kry_4 |   kry_5 |   kry_6 |
|----+--------------+---------+---------+---------+---------|
|  0 |            0 |   10483 |  315972 |   10483 |  226129 |
|  1 |            1 |   10647 |  354146 |   10647 |  253123 |
|  2 |            2 |   10603 |  339736 |   10603 |  237705 |
|  3 |            3 |   10647 |  319468 |   10647 |  211903 |
|  4 |            4 |   10647 |  340688 |   10647 |  240227 |
+----+--------------+---------+---------+---------+---------+
